# 🚀 TensorAeroSpace - Быстрый старт

Добро пожаловать в **TensorAeroSpace** - мощную библиотеку для моделирования и управления аэрокосмическими системами!

## 🎯 Что вы изучите:
1. **Основы библиотеки** - импорты и базовая настройка
2. **Модели летательных аппаратов** - F-16, B-747, UAV и другие
3. **Агенты управления** - IHDP, PID, MPC, RL агенты
4. **Сигналы и утилиты** - генерация опорных сигналов
5. **Бенчмаркинг** - анализ качества управления

---

## 📦 Установка

```bash
pip install tensoraerospace
```

Или для разработки:
```bash
git clone https://github.com/TensorAeroSpace/TensorAeroSpace.git
cd TensorAeroSpace
pip install -e .
```

## 🔧 Импорт основных компонентов

In [ ]:
# Базовые библиотеки
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
from tqdm import tqdm

# TensorAeroSpace - Модели летательных аппаратов
from tensoraerospace.envs import (
    LinearLongitudinalF16,      # Истребитель F-16
    LinearLongitudinalB747,     # Боинг 747
    LinearLongitudinalUAV,      # Беспилотник
    LinearLongitudinalX15,      # Экспериментальный X-15
    GeoSatEnv,                  # Геостационарный спутник
    ComSatEnv                   # Спутник связи
)

# TensorAeroSpace - Агенты управления
from tensoraerospace.agent import (
    IHDPAgent,                  # Incremental Heuristic Dynamic Programming
    PPO,                        # Proximal Policy Optimization
    SAC,                        # Soft Actor-Critic
    AircraftMPC                 # Model Predictive Control
)

# TensorAeroSpace - PID контроллер
from tensoraerospace.agent.pid import PID

# TensorAeroSpace - Сигналы и утилиты
from tensoraerospace.signals import unit_step, sinusoid
from tensoraerospace.signals.random import full_random_signal
from tensoraerospace.utils import generate_time_period, convert_tp_to_sec_tp

# TensorAeroSpace - Бенчмаркинг
from tensoraerospace.benchmark import ControlBenchmark
from tensoraerospace.benchmark.function import (
    overshoot, settling_time, static_error, rise_time
)

# Настройка matplotlib
try:
    plt.style.use('seaborn-v0_8')
except:
    plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12

print('✅ Все компоненты TensorAeroSpace успешно импортированы!')

## ✈️ Демонстрация моделей летательных аппаратов

TensorAeroSpace поддерживает множество моделей:

In [ ]:
# Параметры симуляции
dt = 0.01
simulation_time = 10
tp = generate_time_period(tn=simulation_time, dt=dt)
tps = convert_tp_to_sec_tp(tp, dt=dt)

# Создание опорного сигнала
reference_signal = unit_step(tp, degree=0.1, time_step=1.0)

print(f'🕐 Параметры симуляции:')
print(f'   • Время: {simulation_time} сек')
print(f'   • Шаг: {dt} сек')
print(f'   • Шагов: {len(tp)}')
print(f'   • Опорный сигнал: ступенька 0.1° на 1 сек')

# Доступные модели
models_info = {
    'F-16 Fighting Falcon': 'Многоцелевой истребитель',
    'Boeing 747': 'Широкофюзеляжный пассажирский самолет',
    'UAV': 'Беспилотный летательный аппарат',
    'X-15': 'Экспериментальный гиперзвуковой самолет',
    'GeoSat': 'Геостационарный спутник',
    'ComSat': 'Спутник связи'
}

print('\n🛩️ Доступные модели:')
for model, description in models_info.items():
    print(f'   • {model}: {description}')

## 🎯 Пример 1: Управление F-16 с помощью PID контроллера

In [ ]:
# Создание среды F-16
initial_state = np.array([[0.0], [0.0]])
env_f16 = LinearLongitudinalF16(
    initial_state=initial_state,
    reference_signal=reference_signal,
    number_time_steps=len(tp)
)

# Настройка PID контроллера
pid_controller = PID(
    env=env_f16,
    kp=1.0,    # Пропорциональный коэффициент
    ki=0.1,    # Интегральный коэффициент
    kd=0.05,   # Дифференциальный коэффициент
    dt=dt
)

print('🎮 PID контроллер настроен для F-16')
print(f'   • Kp = {pid_controller.kp}')
print(f'   • Ki = {pid_controller.ki}')
print(f'   • Kd = {pid_controller.kd}')

# Симуляция с PID
state = env_f16.reset()
states_pid = []
controls_pid = []
errors_pid = []

for i in tqdm(range(len(tp)), desc='🚀 Симуляция F-16 + PID'):
    # Управляющий сигнал от PID
    control = pid_controller.select_action(reference_signal[i], state[1])
    
    # Ошибка для записи
    error = reference_signal[i] - state[1]  # alpha (угол атаки)
    
    # Ограничение управления
    control = np.clip(control, -0.5, 0.5)
    
    # Шаг симуляции
    state, _, _, _, _ = env_f16.step([control])
    
    # Сохранение данных
    states_pid.append(state.copy())
    controls_pid.append(control)
    errors_pid.append(error)

states_pid = np.array(states_pid)
controls_pid = np.array(controls_pid)
errors_pid = np.array(errors_pid)

print('✅ Симуляция F-16 + PID завершена!')

## 🧠 Пример 2: Интеллектуальное управление с IHDP агентом

In [ ]:
# Создание новой среды для IHDP
env_ihdp = LinearLongitudinalF16(
    initial_state=initial_state,
    reference_signal=reference_signal,
    number_time_steps=len(tp)
)

# Настройки IHDP агента
actor_settings = {
    'start_training': 100,
    'layers': [4, 64, 32, 1],
    'activations': ['tanh', 'tanh', 'linear'],
    'learning_rate': 0.001,
    'type_PE': 'random',
    'amplitude_3211': 0.1,
    'pulse_length_3211': 100,
    'maximum_input': 0.5,
    'maximum_q_rate': 1.0,
    'WB_limits': [-5, 5],
    'NN_initial': 'random',
    'cascade_actor': False
}

critic_settings = {
    'Q_weights': [1, 1, 1, 1],
    'start_training': 50,
    'gamma': 0.95,
    'learning_rate': 0.001,
    'layers': [8, 64, 32, 1],
    'activations': ['tanh', 'tanh', 'linear'],
    'indices_tracking_states': [1],
    'WB_limits': [-5, 5],
    'NN_initial': 'random'
}

incremental_settings = {
    'number_time_steps': len(tp),
    'dt': dt,
    'input_magnitude_limits': [-0.5, 0.5],
    'input_rate_limits': [-2.0, 2.0]
}

# Создание IHDP агента
ihdp_agent = IHDPAgent(
    env=env_ihdp,
    actor_settings=actor_settings,
    critic_settings=critic_settings,
    incremental_settings=incremental_settings
)

print('🧠 IHDP агент создан и настроен!')
print(f'   • Архитектура актора: {actor_settings["layers"]}')
print(f'   • Архитектура критика: {critic_settings["layers"]}')
print(f'   • Начало обучения актора: {actor_settings["start_training"]} шаг')
print(f'   • Начало обучения критика: {critic_settings["start_training"]} шаг')

In [ ]:
# Симуляция с IHDP
state = env_ihdp.reset()
states_ihdp = []
controls_ihdp = []
rewards_ihdp = []

for i in tqdm(range(len(tp)), desc='🧠 Симуляция F-16 + IHDP'):
    # Получение управления от IHDP агента
    control = ihdp_agent.predict(state, reference_signal[i])
    
    # Шаг симуляции
    next_state, reward, done, truncated, info = env_ihdp.step([control])
    
    # Обучение агента
    ihdp_agent.fit(state, [control], reward, next_state, reference_signal[i])
    
    # Сохранение данных
    states_ihdp.append(state.copy())
    controls_ihdp.append(control)
    rewards_ihdp.append(reward)
    
    state = next_state

states_ihdp = np.array(states_ihdp)
controls_ihdp = np.array(controls_ihdp)
rewards_ihdp = np.array(rewards_ihdp)

print('✅ Симуляция F-16 + IHDP завершена!')
print(f'   • Средняя награда: {np.mean(rewards_ihdp):.4f}')
print(f'   • Общая награда: {np.sum(rewards_ihdp):.2f}')

## 📊 Пример 3: Различные типы сигналов

In [ ]:
# Генерация различных сигналов
step_signal = unit_step(tp, degree=0.15, time_step=2.0)
sine_signal = sinusoid(tps, frequency=0.5, amplitude=0.1)
random_signal = full_random_signal(len(tp), amplitude_range=[-0.1, 0.1])

# Визуализация сигналов
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('📊 Типы опорных сигналов в TensorAeroSpace', fontsize=16, fontweight='bold')

# Ступенчатый сигнал
axes[0, 0].plot(tps, step_signal, 'b-', linewidth=2, label='Ступенчатый сигнал')
axes[0, 0].set_title('🔲 Ступенчатый сигнал')
axes[0, 0].set_xlabel('Время (с)')
axes[0, 0].set_ylabel('Амплитуда (°)')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].legend()

# Синусоидальный сигнал
axes[0, 1].plot(tps, sine_signal, 'r-', linewidth=2, label='Синусоидальный сигнал')
axes[0, 1].set_title('🌊 Синусоидальный сигнал')
axes[0, 1].set_xlabel('Время (с)')
axes[0, 1].set_ylabel('Амплитуда (°)')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].legend()

# Случайный сигнал
axes[1, 0].plot(tps, random_signal, 'g-', linewidth=1, alpha=0.7, label='Случайный сигнал')
axes[1, 0].set_title('🎲 Случайный сигнал')
axes[1, 0].set_xlabel('Время (с)')
axes[1, 0].set_ylabel('Амплитуда (°)')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].legend()

# Комбинированный сигнал
combined_signal = step_signal + 0.5 * sine_signal
axes[1, 1].plot(tps, combined_signal, 'm-', linewidth=2, label='Комбинированный сигнал')
axes[1, 1].set_title('🔀 Комбинированный сигнал')
axes[1, 1].set_xlabel('Время (с)')
axes[1, 1].set_ylabel('Амплитуда (°)')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].legend()

plt.tight_layout()
plt.show()

print('📊 Демонстрация различных типов сигналов завершена!')

## 📈 Сравнение результатов PID vs IHDP

In [ ]:
# Создание сравнительных графиков
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('🎯 Сравнение PID и IHDP управления F-16', fontsize=16, fontweight='bold')

# График 1: Угол атаки (alpha)
axes[0, 0].plot(tps, reference_signal, 'k--', linewidth=2, label='Опорный сигнал', alpha=0.8)
axes[0, 0].plot(tps, states_pid[:, 1], 'b-', linewidth=2, label='PID управление')
axes[0, 0].plot(tps, states_ihdp[:, 1], 'r-', linewidth=2, label='IHDP управление')
axes[0, 0].set_title('📐 Угол атаки (α)')
axes[0, 0].set_xlabel('Время (с)')
axes[0, 0].set_ylabel('Угол атаки (°)')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].legend()

# График 2: Управляющие сигналы
axes[0, 1].plot(tps, controls_pid, 'b-', linewidth=2, label='PID управление')
axes[0, 1].plot(tps, controls_ihdp, 'r-', linewidth=2, label='IHDP управление')
axes[0, 1].set_title('🎮 Управляющие сигналы')
axes[0, 1].set_xlabel('Время (с)')
axes[0, 1].set_ylabel('Управление')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].legend()

# График 3: Ошибки управления
error_ihdp = reference_signal - states_ihdp[:, 1]
axes[1, 0].plot(tps, errors_pid, 'b-', linewidth=2, label='Ошибка PID')
axes[1, 0].plot(tps, error_ihdp, 'r-', linewidth=2, label='Ошибка IHDP')
axes[1, 0].set_title('❌ Ошибки управления')
axes[1, 0].set_xlabel('Время (с)')
axes[1, 0].set_ylabel('Ошибка (°)')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].legend()

# График 4: Награды IHDP
axes[1, 1].plot(tps, rewards_ihdp, 'g-', linewidth=2, label='Награды IHDP')
axes[1, 1].set_title('🏆 Награды IHDP агента')
axes[1, 1].set_xlabel('Время (с)')
axes[1, 1].set_ylabel('Награда')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].legend()

plt.tight_layout()
plt.show()

## 📊 Бенчмаркинг качества управления

In [ ]:
# Создание объекта для бенчмаркинга
benchmark = ControlBenchmark()

# Анализ качества PID управления
pid_metrics = benchmark.becnchmarking_one_step(
    control_signal=controls_pid,
    system_signal=states_pid[:, 1],
    signal_val=0.1,
    dt=dt
)

# Анализ качества IHDP управления
ihdp_metrics = benchmark.becnchmarking_one_step(
    control_signal=controls_ihdp,
    system_signal=states_ihdp[:, 1],
    signal_val=0.1,
    dt=dt
)

# Вывод результатов
print('📊 РЕЗУЛЬТАТЫ БЕНЧМАРКИНГА')
print('=' * 50)

print('🎮 PID Контроллер:')
print(f'   • Перерегулирование: {pid_metrics["overshoot"]:.2f}%')
print(f'   • Время установления: {pid_metrics["settling_time"]:.2f} с')
print(f'   • Статическая ошибка: {pid_metrics["static_error"]:.4f}')
print(f'   • Время нарастания: {pid_metrics["rise_time"]:.2f} с')

print('🧠 IHDP Агент:')
print(f'   • Перерегулирование: {ihdp_metrics["overshoot"]:.2f}%')
print(f'   • Время установления: {ihdp_metrics["settling_time"]:.2f} с')
print(f'   • Статическая ошибка: {ihdp_metrics["static_error"]:.4f}')
print(f'   • Время нарастания: {ihdp_metrics["rise_time"]:.2f} с')

# Дополнительные метрики
pid_mae = np.mean(np.abs(errors_pid))
ihdp_mae = np.mean(np.abs(error_ihdp))

pid_rmse = np.sqrt(np.mean(errors_pid**2))
ihdp_rmse = np.sqrt(np.mean(error_ihdp**2))

print('📈 Дополнительные метрики:')
print(f'   • MAE (PID): {pid_mae:.4f}')
print(f'   • MAE (IHDP): {ihdp_mae:.4f}')
print(f'   • RMSE (PID): {pid_rmse:.4f}')
print(f'   • RMSE (IHDP): {ihdp_rmse:.4f}')

# Определение лучшего контроллера
if ihdp_mae < pid_mae:
    print('🏆 ПОБЕДИТЕЛЬ: IHDP агент показал лучшую точность!')
else:
    print('🏆 ПОБЕДИТЕЛЬ: PID контроллер показал лучшую точность!')

## 🌟 Пример 4: Быстрая демонстрация других моделей

In [ ]:
# Создание различных моделей
models = {
    'Boeing 747': LinearLongitudinalB747(),
    'UAV': LinearLongitudinalUAV(),
    'X-15': LinearLongitudinalX15()
}

# Быстрая симуляция каждой модели
results = {}

for model_name, env in models.items():
    print(f'🚀 Тестирование {model_name}...')
    
    # Простой PID для каждой модели
    pid = PID(env=env, kp=0.5, ki=0.05, kd=0.02, dt=dt)
    
    state = env.reset()
    states = []
    
    # Короткая симуляция (5 секунд)
    short_tp = generate_time_period(tn=5, dt=dt)
    short_ref = unit_step(short_tp, degree=0.05, time_step=1.0)
    
    for i in range(len(short_tp)):
        control = pid.select_action(short_ref[i], state[1])
        control = np.clip(control, -0.3, 0.3)
        state, _, _, _, _ = env.step([control])
        states.append(state[1])  # Сохраняем только alpha
    
    results[model_name] = np.array(states)
    print(f'   ✅ {model_name} - симуляция завершена')

# Визуализация результатов
plt.figure(figsize=(12, 8))
short_tps = convert_tp_to_sec_tp(short_tp, dt=dt)

plt.plot(short_tps, short_ref, 'k--', linewidth=3, label='Опорный сигнал', alpha=0.8)

colors = ['blue', 'red', 'green', 'orange', 'purple']
for i, (model_name, response) in enumerate(results.items()):
    plt.plot(short_tps, response, color=colors[i], linewidth=2, label=f'{model_name}')

plt.title('🛩️ Сравнение откликов различных моделей', fontsize=14, fontweight='bold')
plt.xlabel('Время (с)')
plt.ylabel('Угол атаки (°)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

print('🌟 Демонстрация различных моделей завершена!')

## 🎉 Заключение

Поздравляем! Вы изучили основы **TensorAeroSpace**:

### ✅ Что вы освоили:
1. **Модели летательных аппаратов** - F-16, B-747, UAV, X-15, спутники
2. **Контроллеры** - PID, IHDP, MPC, RL агенты
3. **Сигналы** - ступенчатые, синусоидальные, случайные
4. **Бенчмаркинг** - анализ качества управления
5. **Визуализация** - профессиональные графики результатов

### 🚀 Следующие шаги:
- Изучите папку `examples/` для более сложных примеров
- Попробуйте другие агенты: PPO, SAC, A3C
- Экспериментируйте с различными моделями
- Настройте гиперпараметры для лучшей производительности

### 📚 Полезные ресурсы:
- [Документация](https://tensoraerospace.readthedocs.io/)
- [GitHub репозиторий](https://github.com/TensorAeroSpace/TensorAeroSpace)
- [Примеры использования](https://github.com/TensorAeroSpace/TensorAeroSpace/tree/main/example)

---

**TensorAeroSpace** - ваш надежный инструмент для моделирования и управления аэрокосмическими системами! 🚀

In [ ]:
print('🎯 ИТОГОВАЯ СВОДКА QUICKSTART')
print('=' * 40)
print(f'📊 Симуляций выполнено: 4')
print(f'🛩️ Моделей протестировано: 4')
print(f'🎮 Контроллеров использовано: 2')
print(f'📈 Графиков построено: 3')
print(f'⏱️ Общее время симуляции: {simulation_time + 5} сек')
print('\n🚀 Добро пожаловать в мир TensorAeroSpace!')
print('   Удачных экспериментов! 🌟')